In [20]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, cross_val_score, KFold, StratifiedKFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [21]:
toy_data = pd.read_csv('toy_dataset.csv')
toy_data.head()

,Number,City,Gender,Age,Income,Illness
0,1,Dallas,Male,NaN,40367.0,No
1,2,Dallas,Male,54.0,45084.0,No
2,3,Dallas,Male,NaN,52483.0,No
3,4,Dallas,Male,40.0,40941.0,No
4,5,Dallas,Male,NaN,50289.0,No


In [22]:
toy_data.dtypes

Number       int64
City        object
Gender      object
Age        float64
Income     float64
Illness     object
dtype: object

In [23]:
# проверим есть ли пропущенные значения
toy_data.isnull().sum()

Number     0
City       0
Gender     0
Age        3
Income     0
Illness    0
dtype: int64

In [24]:
# Применение Label Encoding к колонке 'Gender'
label_encoder = LabelEncoder()
toy_data['Gender'] = label_encoder.fit_transform(toy_data['Gender'])
toy_data['City'] = label_encoder.fit_transform(toy_data['City'])
toy_data['Illness'] = label_encoder.fit_transform(toy_data['Illness'])

In [42]:
# проверим есть ли пропущенные значения
toy_data.isnull().sum()

Number     0
City       0
Gender     0
Age        0
Income     0
Illness    0
dtype: int64

In [25]:
cat_temp_data = toy_data[['Age']]
cat_temp_data.head()
cat_temp_data['Age'].unique()
cat_temp_data[cat_temp_data['Age'].isnull()].shape

(3, 1)

In [26]:
imp2 = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
data_imp2 = imp2.fit_transform(cat_temp_data)
data_imp2
toy_data['Age'] = data_imp2

In [27]:
toy_data.head()

,Number,City,Gender,Age,Income,Illness
0,1,2,1,26.0,40367.0,0
1,2,2,1,54.0,45084.0,0
2,3,2,1,26.0,52483.0,0
3,4,2,1,40.0,40941.0,0
4,5,2,1,26.0,50289.0,0


In [28]:
# Разделение данных на обучающую и тестовую выборки
train_data, test_data = train_test_split(toy_data, test_size=0.2, random_state=42)

In [29]:
# Итоговый DataFrame
print("Обучающая выборка:")
print(train_data.head())


Обучающая выборка:
        Number  City  Gender   Age    Income  Illness
104025  104026     4       0  43.0  120945.0        0
5415      5416     2       1  62.0   61173.0        0
75612    75613     3       0  42.0   97477.0        0
138169  138170     0       1  64.0   89428.0        0
87184    87185     3       1  29.0   82193.0        0


In [30]:

print("Тестовая выборка:")
print(test_data.head())

Тестовая выборка:
        Number  City  Gender   Age    Income  Illness
59770    59771     5       1  49.0  102931.0        0
21362    21363     5       1  45.0   99505.0        1
127324  127325     7       1  40.0   67579.0        0
140509  140510     0       1  37.0   90818.0        0
144297  144298     0       1  65.0   89077.0        0


In [31]:
# Разделение данных на признаки и целевую переменную
X = toy_data.drop(['Gender'], axis=1)  # Предполагаем, что 'Illness_No' является целевой переменной
y = toy_data['Gender']

In [32]:
# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [33]:
# Обучение модели k-NN с заданным гиперпараметром k
k = 3
knn = KNeighborsClassifier(n_neighbors=k)
knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=3)

In [34]:
# Предсказание на тестовой выборке
y_pred = knn.predict(X_test)

In [35]:
# Оценка качества модели
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

In [36]:
print(f"Accuracy: {accuracy}")
print("Classification Report:")
print(class_report)

Accuracy: 0.6347333333333334
Confusion Matrix:
[[ 7540  5675]
 [ 5283 11502]]
Classification Report:
              precision    recall  f1-score   support

           0       0.59      0.57      0.58     13215
           1       0.67      0.69      0.68     16785

    accuracy                           0.63     30000
   macro avg       0.63      0.63      0.63     30000
weighted avg       0.63      0.63      0.63     30000



In [37]:
# Подбор гиперпараметра K с использованием GridSearchCV
param_grid = {'n_neighbors': range(1, 20)}
grid_search = GridSearchCV(KNeighborsClassifier(), param_grid, cv=5)
grid_search.fit(X_train, y_train)


GridSearchCV(cv=5, estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': range(1, 20)})

In [38]:
# Оценка качества оптимальной модели GridSearchCV
best_knn_grid = grid_search.best_estimator_
y_pred_grid = best_knn_grid.predict(X_test)
accuracy_grid = accuracy_score(y_test, y_pred_grid)
conf_matrix_grid = confusion_matrix(y_test, y_pred_grid)
class_report_grid = classification_report(y_test, y_pred_grid)

print(f"Accuracy модели GridSearchCV: {accuracy_grid}")
print("Confusion Matrix модели GridSearchCV:")
print(conf_matrix_grid)
print("Classification Report модели GridSearchCV:")
print(class_report_grid)

Accuracy модели GridSearchCV: 0.6776
Confusion Matrix модели GridSearchCV:
[[ 7806  5409]
 [ 4263 12522]]
Classification Report модели GridSearchCV:
              precision    recall  f1-score   support

           0       0.65      0.59      0.62     13215
           1       0.70      0.75      0.72     16785

    accuracy                           0.68     30000
   macro avg       0.67      0.67      0.67     30000
weighted avg       0.68      0.68      0.68     30000



In [39]:
# Подбор гиперпараметра K с использованием RandomizedSearchCV
param_dist = {'n_neighbors': range(1, 20)}
random_search = RandomizedSearchCV(KNeighborsClassifier(), param_dist, n_iter=10, cv=5, random_state=42)
random_search.fit(X_train, y_train)


RandomizedSearchCV(cv=5, estimator=KNeighborsClassifier(),
                   param_distributions={'n_neighbors': range(1, 20)},
                   random_state=42)

In [40]:
# Оценка качества оптимальной модели RandomizedSearchCV
best_knn_random = random_search.best_estimator_
y_pred_random = best_knn_random.predict(X_test)
accuracy_random = accuracy_score(y_test, y_pred_random)
conf_matrix_random = confusion_matrix(y_test, y_pred_random)
class_report_random = classification_report(y_test, y_pred_random)

print(f"Accuracy модели RandomizedSearchCV: {accuracy_random}")
print("Confusion Matrix модели RandomizedSearchCV:")
print(conf_matrix_random)
print("Classification Report модели RandomizedSearchCV:")
print(class_report_random)

Accuracy модели RandomizedSearchCV: 0.6738666666666666
Confusion Matrix модели RandomizedSearchCV:
[[ 7763  5452]
 [ 4332 12453]]
Classification Report модели RandomizedSearchCV:
              precision    recall  f1-score   support

           0       0.64      0.59      0.61     13215
           1       0.70      0.74      0.72     16785

    accuracy                           0.67     30000
   macro avg       0.67      0.66      0.67     30000
weighted avg       0.67      0.67      0.67     30000



In [41]:
# Оценка качества модели с использованием кросс-валидации
kf = KFold(n_splits=5, shuffle=True, random_state=42)
stratified_kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

cross_val_scores_kf = cross_val_score(best_knn_grid, X, y, cv=kf)
cross_val_scores_stratified_kf = cross_val_score(best_knn_grid, X, y, cv=stratified_kf)

print(f"Средняя точность модели с использованием KFold: {cross_val_scores_kf.mean()}")
print(f"Средняя точность модели с использованием StratifiedKFold: {cross_val_scores_stratified_kf.mean()}")

Средняя точность модели с использованием KFold: 0.6789333333333333
Средняя точность модели с использованием StratifiedKFold: 0.6782266666666666
